### 1. 介绍

- **提出背景**：
  - 在图像特征提取和识别等高维情况下，传统岭回归或其扩展方法存在小类问题，即通过岭回归得到的投影数量受限于类的数量。
  - 由于岭回归基于L2，因此对离群值敏感。

- **GRR**：广义鲁棒回归（Generalized Robust Regression, GRR），用于联合稀疏子空间学习，具体实现包括：
    
  - **L2,1范数惩罚**：对损失函数和正则化项施加L2,1范数惩罚，以保证联合稀疏性和对离群值的鲁棒性。
  - **内在局部几何结构**：利用L2,1范数作为度量，考虑数据的内在局部几何结构以提高性能。
  - **弹性因子**：通过在损失函数上引入弹性因子，增强鲁棒性，获得更多用于特征选择或分类的投影。
  
- **GRR损失函数的几何意义（★★★）**：最小重构误差下，得到联合稀疏，具有正交性，松弛的特征，同时维持数据的局部几何结构，有较强的鲁棒性。
  - 最小重构误差下
  - 联合稀疏，具有正交性，松弛的特征
  - 维持数据的局部几何结构
  

- **优点**：GRR在联合稀疏子空间学习中具有更好的性能，对离群值具有更好的鲁棒性。

- **目标函数**：GRR 的目标函数是最小化基于 $L_{2,1}$-范数的优化问题，并具有一些约束：

  $$
  \begin{aligned}
  &\min_{A,B,h} && \sum_{i} \sum_{j} \left\|x_{i}BA^{T} - x_{j}BA^{T}\right\|_{2}W_{ij} + \beta \left\|B\right\|_{2,1} \\
  &&& + \gamma \left\|XBA^{T} + 1h^{T} - Y\right\|_{2,1} + \lambda \left\|h\right\|_{2}^{2} \\
  &s.t.\: && A^{T}A = I
  \end{aligned}
  $$

  其中 $B \in \mathbb{R}^{d \times k}$ 是投影矩阵，$A \in \mathbb{R}^{c \times k}$ 是正交矩阵，$d$ 和 $k$ 是矩阵的尺寸，而 $c$ 是类别数。$W \in \mathbb{R}^{n \times n}$ 是LPP中定义的相似性矩阵。$1 \in \mathbb{R}^{n \times 1}$ 是所有元素都为 1 的向量。向量 $h \in \mathbb{R}^{c \times 1}$ 是偏置项，三个系数 $\beta, \gamma$ 和 $\lambda$ 是平衡不同项的参数。注意，偏置项 $h$ 在一些先前的半监督算法中使用过，例如 LapRLS/L [32], FME [33] 等。所提出的方法将该偏置项扩展为广义回归中的弹性因子，以提高人脸识别的鲁棒性，特别是在数据点被强噪声污染时。

  - **几何意义（★★★）**：最小重构误差下，得到联合稀疏，具有正交性，松弛的特征，同时维持数据的局部几何结构，有较强的鲁棒性。

- **推导步骤（省流）**:
  
  1. 根据LLP的方法，消去$\sum$变成矩阵形式，注意A是正交的，因此被消去，而B在外面。
  2. 然后把$\|·\|_2$变成$\|·\|_2^2$，注意分子分母乘以$·$后，令分母的$·$变成$G_{i,j}$，挨在$W_{i,j}$下面
  3. $\emptyset$表示两矩阵对应位置的元素相乘运算。
  4. $\|·\|_2$变成$tr(B^{T}X^{T}(D-W\emptyset G)XB)$，其中$D$是$W\emptyset G$按照列求和得到的对角矩阵。
  5. 分别对$h$和$B$求导，固定$h$和$B$后，目标函数变成最小化A的展开一次项（负的），也就是最大化这个项
  6. 用SVD分解定理：$$\hat{S} = \arg \max_{S} \operatorname{tr}(S^{T} Z) \quad \text{s.t.} \quad S^{T}S=I_k$$假设 $Z$ 的 SVD 为 $Z=\check{U}\check{D}\check{V}^{T}$，则 $\hat{S}=\check{U}\check{V}^{T}$，得到A的解。




- **推导流程**:

  对于目标函数中的第一部分，由于我们将欧几里得范数的平方改为 $L_{2,1}$-范数，因此为了利用 LPP 的性质，我们将其重新表述如下：
  $$\sum_{i}\sum_{j}\left\|x_{i}BA^{T}-x_{j}BA^{T}\right\|_{2}W_{ij} = \sum_i\sum_j \frac{\left\|x_iBA^T-x_jBA^T\right\|_2^2}{\left\|x_iBA^T-x_jBA^T\right\|_2}W_{ij} = \sum_i\sum_j \left\|x_iBA^T-x_jBA^T\right\|_2^2 \frac{W_{ij}}{G_{ij}}$$
  其中 $G_{ij} = \left\|x_{i}BA^{T}-x_{j}BA^{T}\right\|_{2}$。因此，我们有：
  $$\sum_{i}\sum_{j}\left\|x_{i}BA^{T}-x_{j}BA^{T}\right\|_{2}W_{ij} = \operatorname{tr}(B^{T}X^{T}(D-W\emptyset G)XB)$$
  其中 $\emptyset$ 表示矩阵的逐元素相乘，$D$ 是一个对角矩阵，其元素是 $W\emptyset G$ 的行（或列）和，即 $D_{ii}=\sum_{j}(W\emptyset G)_{ij}$。
  因此目标函数等价于以下函数：
  $$\begin{aligned}
  &\min_{A,B,h} \operatorname{tr}(B^{T}X^{T}(D-W\emptyset G)XB) + \beta \operatorname{tr}(B^{T}\tilde{D}B) \\
  &+ \gamma \operatorname{tr}((XBA^{T}+1h^{T}-Y)^{T}\hat{D}(XBA^{T}+1h^{T}-Y)) \\
  &+ \lambda \operatorname{tr}(h^{T}h) \\
  &\text{s.t.} \quad A^{T}A=I
  \end{aligned}$$
  为了获得 GRR 的局部最优解，我们固定两个变量 $A$ 和 $B$，并将 (9) 对 $h$ 的导数设为零，则有：

  $$h = \frac{1}{s}(Y^T \hat{D} 1 - AB^T X^T \hat{D} 1)$$
  其中 $s = 1^{T} \hat{D} 1 + \lambda I$。

  类似地，对于固定的 $A$ 和 $h$，我们将 (9) 对 $B$ 的导数设为零，则 (9) 通过以下公式最小化：
  $$B = \gamma \left[\beta \tilde{D} + X^{T} ((D-W\emptyset G) + \gamma \hat{D}) X\right]^{-1} X^{T} \hat{D} (Y - 1h^{T}) A$$
  当两个变量 $B$ 和 $h$ 固定时，关于 $A$ 的以下最大化问题提供了最优解：
  
  $$\max_{A} \operatorname{tr}(A^{T} (h \mathbf{1}^{T} - Y^{T}) \hat{D} XB) \quad \text{s.t.} \quad A^{T}A=I$$

  最优解可以从以下定理中获得：

  **定理**：设 $S$ 和 $Z$ 是 $c \times k$ 矩阵，且 $Z$ 的秩为 $k$。给定以下优化问题：
  $$\hat{S} = \arg \max_{S} \operatorname{tr}(S^{T} Z) \quad \text{s.t.} \quad S^{T}S=I_k$$
  假设 $Z$ 的 SVD 为 $Z=\check{U}\check{D}\check{V}^{T}$，则 $\hat{S}=\check{U}\check{V}^{T}$。

  因此对于给定的 $B$ 和 $h$，(12) 中 $A$ 的 SVD 为 $(h \mathbf{1}^{T} - Y^{T}) \hat{D} XB = U \bar{D} V^{T}$，则
  
  $$A = UV^{T}$$

### 2. 摘译

#### 2.1. 摘要

岭回归广泛应用于多变量数据分析。然而，在图像特征提取和识别等高维情况下，传统岭回归或其扩展方法存在小类问题，即通过岭回归得到的投影数量受限于类的数量。本文提出了一种新方法，称为广义鲁棒回归（Generalized Robust Regression, GRR），用于联合稀疏子空间学习，可以解决这个问题。GRR不仅对损失函数和正则化项施加L2,1范数惩罚，以保证联合稀疏性和对离群值的鲁棒性，实现有效的特征选择，还利用L2,1范数作为度量，考虑数据的内在局部几何结构以提高性能。此外，通过在损失函数上引入弹性因子，GRR可以增强鲁棒性，获得更多用于特征选择或分类的投影。为了获得GRR的最优解，提出了一种迭代算法，并证明了其收敛性。对六个著名数据集的实验展示了所提方法的优点。结果表明，GRR是一种用于人脸识别的鲁棒且高效的回归方法。


#### 2.2. 广义鲁棒回归

本节将介绍一种用于联合稀疏子空间学习的模型，称为广义鲁棒回归（Generalized Robust Regression，GRR），并设计交替迭代算法来解决优化问题。



##### A. 记号

标量表示为小写斜体字母，如 $i, j, n, d, c, m$ 等。向量表示为粗体小写斜体字母，如 $x, y, v$ 等。矩阵定义为粗体大写斜体字母，如 $X, Y, A, B$ 等。

- 设 $X \in \mathbb{R}^{n \times d}$ 表示训练样本矩阵，其中 $n$ 是训练样本的总数，$d$ 表示每个样本的特征维度，$X$ 的每一行是一个样本 $x_i$。
- 设 $Y \in \mathbb{R}^{n \times c}$ 表示标签矩阵，其中 $c$ 是类别的总数。矩阵 $Y$ 被定义为二进制标签矩阵，当 $x_i$ 属于第 $j$ 类时，$Y_{ij} = 1$；否则 $Y_{ij} = 0$。



##### B. 动机和新定义

最小二乘回归（LSR）[1] 和岭回归（RR）[4] 存在一些明显的缺点。首先，它们有小类问题，即当训练数据中的类别数量太少时，仅能获得 $c$ 个投影进行特征提取和选择。其次，由于 LSR 和 RR 是基于 $L_2$ 范数的方法，目标函数上的平方运算会导致对离群值的敏感性。第三，由于传统岭回归学习到的投影不是稀疏的，因此从 LSR 或 RR 学习到的投影在特征选择上没有稀疏性。因此，需要一种更鲁棒的损失函数来进行特征选择。Nie 等人提出了一种通过联合 $L_{2,1}$-范数最小化的有效且鲁棒的特征选择方法（RFS）[28]。RFS 的目标函数为：

$$
\min_P \|XP - Y\|_{2,1} + \gamma \|P\|_{2,1}
$$

通过在损失函数和正则化项上使用 $L_{2,1}$-范数，RFS 能够减少岭回归对离群值的敏感性。此外，RFS 中的正则化项保证了联合稀疏性，从而提高了特征选择和人脸识别的性能。但是，RFS 仍然存在一些缺点。首先，它仍然有小类问题。其次，由于 RFS 学习到的投影仅是数据点全局结构的线性组合，数据集的局部几何结构被忽略。然而，许多实验结果表明，保持局部性倾向于提高特征提取和分类的性能[14]。因此，需要新的技术来解决这些问题。

本文提出了一种用于联合稀疏子空间学习的广义鲁棒回归方法。该方法不仅继承了 RFS 的优点，还结合了 LPP 和 RR 的特性，以进一步提高特征选择的性能。具体来说，它在损失函数上使用 $L_{2,1}$-范数以最小化平方运算误差，同时在正则化项上使用 $L_{2,1}$-范数最小化以保证联合稀疏性。此外，它解决了小类问题，同时考虑了数据的局部几何结构。它还在局部保持投影项上施加 $L_{2,1}$-范数惩罚以确保对离群值的鲁棒性。此外，为了提高所提方法的鲁棒性，弹性因子被引入到损失函数中以进行联合稀疏子空间学习。

##### C. GRR的目标函数
GRR 的目标函数是最小化基于 $L_{2,1}$-范数的优化问题，并具有一些约束：

$$
\begin{aligned}
&\min_{A,B,h} && \sum_{i} \sum_{j} \left\|x_{i}BA^{T} - x_{j}BA^{T}\right\|_{2}W_{ij} + \beta \left\|B\right\|_{2,1} \\
&&& + \gamma \left\|XBA^{T} + 1h^{T} - Y\right\|_{2,1} + \lambda \left\|h\right\|_{2}^{2} \\
&s.t.\: && A^{T}A = I
\end{aligned}
$$

其中 $B \in \mathbb{R}^{d \times k}$ 是投影矩阵，$A \in \mathbb{R}^{c \times k}$ 是正交矩阵，$d$ 和 $k$ 是矩阵的尺寸，而 $c$ 是类别数。$W \in \mathbb{R}^{n \times n}$ 是LPP中定义的相似性矩阵。$1 \in \mathbb{R}^{n \times 1}$ 是所有元素都为 1 的向量。向量 $h \in \mathbb{R}^{c \times 1}$ 是偏置项，三个系数 $\beta, \gamma$ 和 $\lambda$ 是平衡不同项的参数。注意，偏置项 $h$ 在一些先前的半监督算法中使用过，例如 LapRLS/L [32], FME [33] 等。所提出的方法将该偏置项扩展为广义回归中的弹性因子，以提高人脸识别的鲁棒性，特别是在数据点被强噪声污染时。


在（2）中，第一部分 $\sum_{i}\sum_{j}\left\|x_{i}BA^{T}-x_{j}BA^{T}\right\|_{2}W_{ij}$ 旨在保持局部性[13]。与计算每个训练样本 $x_i (i=1,2,\ldots,n)$ 和 $x_j (j=1,2,\ldots,n)$ 之间的欧几里得距离不同，该距离对离群值敏感，而保持局部信息，所提出的方法使用 $L_{2,1}$-范数作为度量，以增强局部保持能力的鲁棒性。通过继承局部保持特性，所提出的 GRR 不仅保持了数据的内在局部几何结构[13]，还保证了对离群值的鲁棒性。

（2）中的第二部分是正则化项 $\beta\left\|\boldsymbol{B}\right\|_{2,1}$，它保证了学习到的投影矩阵 $B$ 的联合稀疏性，以便进行区分性特征选择[28],[31]。联合稀疏性确保了大多数学习到的投影的元素为零，并选择重要特征进行特征提取。

在（2）中，第三部分 $\gamma\left\|XBA^T+\mathbf{1}h^T-Y\right\|_{2,1}$ 是经典 RR 中的损失函数，而第四部分 
$\lambda\|h\|_{2}^{2}$ 作为偏置项，以保证整个模型的稳定性。与 RR 相比，（2）使用 $L_{2,1}$-范数最小化损失函数，使得模型对离群值更具鲁棒性[28]。GRR 鲁棒性的另一个潜在原因是损失函数上的弹性因子 $h$ 可以在实践中避免过拟合问题。在 RFS 的损失函数上，矩阵 $P$ 必须始终适应 $Y$，以确保矩阵 $XP$ 和标签矩阵 $Y$ 之间的误差最小化，这会导致潜在的过拟合问题。然而，GRR 将弹性因子 $h$ 作为损失函数上的补充项，矩阵 $XBA^T$ 不需要严格适应矩阵 $Y$，以释放过拟合问题，保证特征选择或提取的强泛化能力，特别是在图像被块缺失或噪声污染的情况下。

此外，通过在损失函数上使用矩阵 $BA^{T}$ 而不是 (1) 中的 $P$，（2）旨在解决 LSR、RR 和 RFS 中的小类问题。即 $P$ 的尺寸为 $d \times c$，而 $B, A$ 的尺寸分别为 $d \times k$ 和 $c \times k$，则 $BA^{T}$ 的尺寸为 $d \times c$（即 $BA^{T}$ 的尺寸与 $P$ 相同）。在 LSR、RR 和 RFS 中，投影矩阵为 $P$，学习到的投影数量为 $c$（即类别数量）。然而，在 GRR 中，学习到的投影矩阵为 $B$，其尺寸为 $d \times k$，$k$ 可以设置为任何整数，以获得足够的投影进行人脸识别。因此，所提 GRR 中的投影数量不受类别数量的限制，可以解决 RR 中的小类问题。




##### D. 最优解
根据投影矩阵 $B$ 上 $L_{2,1}$-范数的定义，可以定义对角矩阵 $\tilde{D}$，其第 $i$ 个对角元素如下 [28]：
$$\tilde{D}_{ii} = \frac{1}{2\left\|B^i\right\|_2}$$
其中 $B^{i}$ 表示矩阵 $B$ 的第 $i$ 行。因此，(2) 式中的第二部分可以重写为：
$$\|B\|_{2,1} = \operatorname{tr}(B^T \tilde{D} B)$$
同样地，(2) 式中的第三部分可以重写为：
$$\left\|XBA^{T}+1h^{T}-Y\right\|_{2,1} = \operatorname{tr}((XBA^{T}+1h^{T}-Y)^{T}\hat{D}(XBA^{T}+1h^{T}-Y))$$
其中 $\hat{D}$ 也是一个对角矩阵，其第 $i$ 个对角元素为：
$$\hat{D}_{ii} = \frac{1}{2\left\|(XBA^T+1h^T-Y)^i\right\|_2}$$
对于 (2) 式中的第一部分，由于我们将欧几里得范数的平方改为 $L_{2,1}$-范数，因此为了利用 LPP 的性质，我们将其重新表述如下：
$$\sum_{i}\sum_{j}\left\|x_{i}BA^{T}-x_{j}BA^{T}\right\|_{2}W_{ij} = \sum_i\sum_j \frac{\left\|x_iBA^T-x_jBA^T\right\|_2^2}{\left\|x_iBA^T-x_jBA^T\right\|_2}W_{ij} = \sum_i\sum_j \left\|x_iBA^T-x_jBA^T\right\|_2^2 \frac{W_{ij}}{G_{ij}}$$
其中 $G_{ij} = \left\|x_{i}BA^{T}-x_{j}BA^{T}\right\|_{2}$。因此，我们有：
$$\sum_{i}\sum_{j}\left\|x_{i}BA^{T}-x_{j}BA^{T}\right\|_{2}W_{ij} = \operatorname{tr}(B^{T}X^{T}(D-W\emptyset G)XB)$$
其中 $\emptyset$ 表示矩阵的逐元素相乘，$D$ 是一个对角矩阵，其元素是 $W\emptyset G$ 的行（或列）和，即 $D_{ii}=\sum_{j}(W\emptyset G)_{ij}$。
从 (4)、(5) 和 (8) 得到，目标函数 (2) 等价于以下函数：
$$\begin{aligned}
&\min_{A,B,h} \operatorname{tr}(B^{T}X^{T}(D-W\emptyset G)XB) + \beta \operatorname{tr}(B^{T}\tilde{D}B) \\
&+ \gamma \operatorname{tr}((XBA^{T}+1h^{T}-Y)^{T}\hat{D}(XBA^{T}+1h^{T}-Y)) \\
&+ \lambda \operatorname{tr}(h^{T}h) \\
&\text{s.t.} \quad A^{T}A=I
\end{aligned}$$
为了获得 GRR 的局部最优解，我们固定两个变量 $A$ 和 $B$，并将 (9) 对 $h$ 的导数设为零，则有：
$$h = \frac{1}{s}(Y^T \hat{D} 1 - AB^T X^T \hat{D} 1)$$
其中 $s = 1^{T} \hat{D} 1 + \lambda I$。
类似地，对于固定的 $A$ 和 $h$，我们将 (9) 对 $B$ 的导数设为零，则 (9) 通过以下公式最小化：
$$B = \gamma \left[\beta \tilde{D} + X^{T} ((D-W\emptyset G) + \gamma \hat{D}) X\right]^{-1} X^{T} \hat{D} (Y - 1h^{T}) A$$
在 (9) 中，当两个变量 $B$ 和 $h$ 固定时，关于 $A$ 的以下最大化问题提供了最优解：
$$\max_{A} \operatorname{tr}(A^{T} (h \mathbf{1}^{T} - Y^{T}) \hat{D} XB) \quad \text{s.t.} \quad A^{T}A=I$$
(12) 中的最优解可以从以下定理中获得：
定理 1 [5]：设 $S$ 和 $Z$ 是 $c \times k$ 矩阵，且 $Z$ 的秩为 $k$。给定以下优化问题：
$$\hat{S} = \arg \max_{S} \operatorname{tr}(S^{T} Z) \quad \text{s.t.} \quad S^{T}S=I_k$$
假设 $Z$ 的 SVD 为 $Z=\check{U}\check{D}\check{V}^{T}$，则 $\hat{S}=\check{U}\check{V}^{T}$。
根据定理 1，我们可以知道，对于给定的 $B$ 和 $h$，(12) 中 $A$ 的 SVD 为 $(h \mathbf{1}^{T} - Y^{T}) \hat{D} XB = U \bar{D} V^{T}$，则
$$A = UV^{T}$$
为了获得目标函数的局部最优解，迭代算法的详细步骤如表下所示：

##### E. 算法流程

##### 输入：
- 训练数据 $x \in \mathbb{R}^{n \times d}$
- 训练数据标签 $y \in \mathbb{R}^{n \times c}$
- 对称矩阵 $w \in \mathbb{R}^{n \times n}$
- 目标维度 $k (k=1,2,\ldots,d)$
- 参数 $\beta, \gamma$ 和 $\lambda$
- 最大迭代次数：$maxStep$
##### 输出：
低维判别子空间 $B \in \mathbb{R}^{d \times \hbar}$, $k = 1, 2, \ldots, d$
1. 随机初始化 $A \in \mathbb{R}^{c \times k}, B \in \mathbb{R}^{d \times k}, \tilde{D} \in \mathbb{R}^{d \times d}, \hat{D} \in \mathbb{R}^{n \times n}, h \in \mathbb{R}^{c \times 1}$，初始化 $1 \in \mathbb{R}^{n}$，每个元素为1。
   - 计算 $G \in \mathbb{R}^{n \times n}, D \in \mathbb{R}^{n \times n}$。
   - 设置 step = 1，converged = false。
2. **While** $\sim$ converged && step <= maxStep:
   - 使用 $h = \frac{1}{2}(Y^{T} \hat{D}1 - AB^{T} X^{T} \hat{D}1)$ 计算 $h$，其中 $s = 1^{T} \hat{D} 1 + \lambda I$。
   - 使用 $B = \gamma [\beta \tilde{D} + X^{T}((D - W \otimes G) + \gamma \hat{D})X]^{-1}X^{T} \hat{D} (Y - 1h^{T})A$ 计算 $B$。
   - 使用 (13) 中的 $A = UV^{T}$ 计算 $A$。
   - 使用 $G_{ij} = \left\|x_{i}BA^{T} - x_{j}BA^{T}\right\|_{2}$ 更新 $G$。
   - 使用 $D_{ii} = \sum_{j} (W \otimes G)_{ij}$ 更新 $D$。
   - 使用 $\tilde{D}_{ii} = \frac{1}{2\left\|B^{i}\right\|_{2}}$ 更新 $\tilde{D}$。
   - 使用 $\hat{D}_{ii} = \frac{1}{2\left\|(XBA^{T} + 1h^{T} - Y)^{i}\right\|_{2}}$ 更新 $\hat{D}$。
   - 设置 step = step + 1。
   - 当 $s$ 几乎不变时，更新 converged = true。
3. 标准化矩阵 $s$ 为最终归一化矩阵，并返回以进行特征选择。
##### 说明：
- 该算法使用了一系列矩阵操作来逐步逼近最优解。
- 通过反复迭代和更新权重矩阵以及相关变量，最终得到一个低维判别子空间，用于有效的特征选择。